# Association between Fertility & Local Municipal Characteristics in Denmark 

### Imports

In [20]:
import geopandas as gpd # for geospatial data handling
import pandas as pd
import contextily as cx # for plotting
import matplotlib.pyplot as plt # for plotting
from pyproj import CRS # for more advanced CRS modifications and transformations

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

### Data

Load the data from DenmarkStatistik and geometries for municipalities of Denmark and then merge them into one geo dataframe

'Data/Educational_attainment_by_highest_education_completed_(total)_2021.xlsx'


In [21]:
data_edu = pd.read_excel('Data/Educational_attainment_by_highest_education_completed_(total)_2021.xlsx')
data_fer = pd.read_excel('Data/fertility_rates_by_municipality_2021.xlsx')
data_bir = pd.read_excel('Data/live_births_by_municipality_2021.xlsx')
data_mar = pd.read_excel('Data/market_value_for_real_estate_average_dkk_2021.xlsx')

In [22]:
geometries = gpd.read_file('Data/municipalities_dk.gpkg') #added
# geometry.head()

In [23]:
# rename the columns to prepare for merge
data_edu.rename(columns = {'Unnamed: 0':'municipality_nm', 2021:'edu_att'}, inplace = True)
data_fer.rename(columns = {'Unnamed: 0':'municipality_nm', 2021:'fer_rate'}, inplace = True)
data_bir.rename(columns = {'Unnamed: 0':'municipality_nm', 2021:'live_birth'}, inplace = True)
data_mar.rename(columns = {'Unnamed: 0':'municipality_nm', 2021:'market_val'}, inplace = True)

In [24]:
# merge to one data frame
data = pd.merge(pd.merge(data_fer, data_bir), pd.merge(data_edu, data_mar))
# data.head()

In [25]:
data['municipality_nm'] = data['municipality_nm'].replace('Copenhagen', 'København')
geometries = geometries.rename(columns={'navn': 'municipality_nm'}) #added

,kommunekode,municipal_id
0,0370,370
1,0330,330
2,0766,766
3,0510,510
4,0550,550
...,...,...
94,0851,851
95,0825,825
96,0411,411
97,0779,779


In [38]:
data = pd.merge(data, geometries[['municipality_nm', 'kommunekode', 'municipal_id']]) #added

In [39]:
gdf = gpd.GeoDataFrame(data, geometry=geometries.geometry, crs = 'EPSG:25832') #added

## Inspecting spatial data and EDA

## Spatial Weights

In [31]:
import seaborn as sns
sns.set_theme()
from pysal.lib import weights
from libpysal.io import open as psopen
import numpy as np
import matplotlib.pyplot as plt
from splot.libpysal import plot_spatial_weights

In [35]:
gdf.info

<bound method DataFrame.info of     municipality_nm fer_rate live_birth  edu_att market_val  \
0         København  1 563.9     10 285  501 212  2 399 736   
1     Frederiksberg  1 787.8      1 534   74 788  3 435 510   
2            Dragør  2 510.9        146    9 074  4 014 690   
3            Tårnby  2 056.7        473   28 727  2 823 345   
4       Albertslund  1 608.7        274   18 664  2 931 561   
..              ...      ...        ...      ...        ...   
93            Morsø  2 080.7        180   13 103  1 321 820   
94           Rebild  2 400.5        380   20 166  1 997 252   
95          Thisted  2 063.8        410   28 630  1 422 277   
96  Vesthimmerlands  1 824.4        314   24 005  1 495 512   
97          Aalborg  1 604.3      2 488  158 831  2 042 461   

                                             geometry  
0   MULTIPOLYGON (((672902.450 6143708.870, 672880...  
1   MULTIPOLYGON (((654675.250 6154223.210, 654636...  
2   MULTIPOLYGON (((563379.450 6175110.360,

In [32]:
w_queen = weights.Queen.from_dataframe(gdf, idVariable="fer_rate")

ValueError: The argument to the ids parameter contains duplicate entries.